## Overview

Hello! Welcome to the architecture setup notebook, where we will be installing all requirements and outline the basic architecture of our AlexNet model (whose performance will be compared to our custom model, EfficentNet, and ConvNeXt). 


The cell below handles our initial requirements installation:

In [3]:
!pip3 install -r ../../requirements.txt

Defaulting to user installation because normal site-packages is not writeable


## Data Preprocessing

As part of our data preprocessing, we will split the down-scaled lung dataset from the original dataset into a train/test split. 

Note that we will be using five-fold cross-validation for testing later, hence we will not be partioning an additional validation set. 

After splitting our data, we will then feed the training set into our models. Here, we will specifically feed it into the AlexNet model. 

In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, Subset
import torchvision.transforms as transforms
from torch.utils.data import SubsetRandomSampler
from torchvision.datasets import ImageFolder
from sklearn.model_selection import KFold

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix
import os

from sklearn.decomposition import PCA

import joblib

The code below extracts images from our dataset, resizes each into a fourth their original size (768 -> 192), and converts them into Torch tensors. The ImageFolder class allows us to lazyload our images to preserve our computational power.

In [5]:
# Check current device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Path to our lung_image_sets
data_dir = "../../lung_colon_image_set/lung_image_sets"

# Define resized size of images (Put this back to 192 later, recommended size of 224)
resized_size = 224

# Convert images into Tensors
tensor_data = transforms.Compose([
  transforms.Resize((resized_size, resized_size)),   # Cut image into a fourth of original size
  transforms.Grayscale(num_output_channels=1),      # Convert to grayscale
  transforms.ToTensor(),
  transforms.Lambda(lambda x: x.repeat(3, 1, 1))    # Duplicate channels to get 3 channel image
])

# Load the dataset using ImageFolder
data = ImageFolder(root=data_dir, transform=tensor_data)

# Split the dataset into train and test sets
train_size = int(0.8 * len(data))
test_size = len(data) - train_size
train, test = torch.utils.data.random_split(data, [train_size, test_size])

# Create KFold object with 5 folds
num_folds = 2
kfold = KFold(n_splits=5, shuffle=True, random_state=231)

## Load in pretrained models if possible

If we've already trained classifiers, load them in.

In [6]:
# svm_classifier = None
# softmax_classifier = None
# pca_classifier = None

try:
    svm_classifier = joblib.load('effnet_svm.pkl')
    # svm_tester_classifier = joblib.load('effnet_svm.pkl')
    print('Loaded EfficientNet SVM classifier into svm_tester_classifier')
except:
    print('Could not import EfficientNet SVM classifier, file may not exist.', )
    # svm_classifier = None

try:
    softmax_classifier = joblib.load('effnet_softmax.pkl')
    print('Loaded EfficientNet Softmax classifier into softmax_classifier')
except:
    print('Could not import EfficientNet Softmax classifier, file may not exist.')
    softmax_classifier = None

try:
    pca_classifier = joblib.load('effnet_pca.pkl')
    print('Loaded EfficientNet SVM+PCA classifier into pca_classifier')
except:
    print('Could not import EfficientNet SVM+PCA classifier, file may not exist.')
    pca_classifier = None

Loaded EfficientNet SVM classifier into svm_tester_classifier
Loaded EfficientNet Softmax classifier into softmax_classifier
Loaded EfficientNet SVM+PCA classifier into pca_classifier


## Model Initialization
We will initialize the EfficientNetB0 model using Pytorch's Torchvision pretrained EfficientNetB0 model and remove the final layer to perform feature extraction on our data.

In [7]:
from torchvision.models import efficientnet_b0

# Define the ConvNeXtCNN model
class EfficientNetCNN(nn.Module):
    def __init__(self):
        super(EfficientNetCNN, self).__init__()
        
        # Load the pre-trained EfficientNet-B4 model as our feature extractor
        self.efficientnet = efficientnet_b0(pretrained=True)
        
        # Remove the last layer and freeze the model
        self.efficientnet.classifier = nn.Sequential(*list(self.efficientnet.classifier.children())[:-1])
        self.efficientnet.classifier.eval()

    def forward(self, x):
        # Pass the input through the ConvNeXt model
        # Pass the input through the EfficientNet model
        x = self.efficientnet(x)
        return x
        
# Define instance of our ConvNeXt model
model = EfficientNetCNN()

/home/ubuntu/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/ubuntu/.local/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### EfficientNetB0 + SVM
For our first situation, we will use SVM to do classification on our extracted features.

In [21]:
# Define our SVM classifier
svm_classifier = make_pipeline(StandardScaler(), SVC(kernel='linear'))

# Construct results dict to track training
results = {}

# K-Fold Cross Validation
for fold, (train_indices, val_indices) in enumerate(kfold.split(train), 1):
    print(f'Fold {fold}')

    # Create data samplers for train and validation sets
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    # Create data loaders for train and validation sets
    train_loader = DataLoader(train, batch_size=32, sampler=train_sampler)
    val_loader = DataLoader(train, batch_size=32, sampler=val_sampler)
    
    # Extract features and labels for the training set using ConvNeXt
    train_features = []
    train_labels = []
    with torch.no_grad():
        for inputs, labels in train_loader:
            outputs = model.forward(inputs)
            outputs = outputs.view(outputs.size(0), -1)
            train_features.append(outputs.cpu().numpy())
            train_labels.append(labels.cpu().numpy())
    train_features = np.concatenate(train_features)
    train_labels = np.concatenate(train_labels)

    # Train the SVM classifier
    svm_classifier.fit(train_features, train_labels)
    
    # Extract features and labels for the validation set using ConvNeXt
    val_features = []
    val_labels = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model.forward(inputs)
            outputs = outputs.view(outputs.size(0), -1)
            val_features.append(outputs.cpu().numpy())
            val_labels.append(labels.cpu().numpy())
    val_features = np.concatenate(val_features)
    val_labels = np.concatenate(val_labels)

    # Evaluate the classifier on the validation set and extract metrics
    val_predictions = svm_classifier.predict(val_features)
    accuracy = accuracy_score(val_labels, val_predictions)

    results[fold] = accuracy
    print(f'Fold {fold} Accuracy: {accuracy:.4f}')

# Print the average metrics across all folds
average_accuracy = np.mean(list(results.values()))

print(f'\nK-FOLD CROSS VALIDATION RESULTS FOR {num_folds} FOLDS')
print('--------------------------------')
for fold in results:
    print(f'Fold {fold}: {results[fold]:.4f}')
print(f'Average: {average_accuracy:.4f}')

Fold 1
Fold 1 Accuracy: 0.9121
Fold 2
Fold 2 Accuracy: 0.9167
Fold 3
Fold 3 Accuracy: 0.9213
Fold 4
Fold 4 Accuracy: 0.9175
Fold 5
Fold 5 Accuracy: 0.9254

K-FOLD CROSS VALIDATION RESULTS FOR 5 FOLDS
--------------------------------
Fold 1: 0.9121
Fold 2: 0.9167
Fold 3: 0.9213
Fold 4: 0.9175
Fold 5: 0.9254
Average: 0.9186


In [26]:
joblib.dump(svm_classifier, 'effnet_svm.pkl')

['effnet_svm.pkl']

## EfficientNetB0 + Softmax Classifier Training and Testing
We will perform k-fold cross-validation testing on the Softmax classifier, which is trained the on features extracted by our EfficientNetB0 model.

In [10]:
# Construct results dict to track training
results = {}

# Create the softmax classifier pipeline via a Logistic Regression with Softmax activation.
softmax_classifier = make_pipeline(
    StandardScaler(),
    LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=400, C=1.0, random_state=231)
)

# K-Fold Cross Validation
for fold, (train_indices, val_indices) in enumerate(kfold.split(train), 1):
    print(f'Fold {fold}')

    # Create data samplers for train and validation sets
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    # Create data loaders for train and validation sets
    train_loader = DataLoader(train, batch_size=32, sampler=train_sampler)
    val_loader = DataLoader(train, batch_size=32, sampler=val_sampler)

    # Extract features and labels for the training set using ConvNeXt
    features = []
    labels = []
    with torch.no_grad():
        for inputs, targets in train_loader:
            outputs = model.forward(inputs)
            outputs = outputs.view(outputs.size(0), -1)
            features.append(outputs.cpu().numpy())
            labels.append(targets.numpy())
    features = np.concatenate(features)
    labels = np.concatenate(labels)

    # Train the softmax classifier
    softmax_classifier.fit(features, labels)

    # Extract features and labels for the validation set using ConvNeXt
    val_features = []
    val_labels = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model.forward(inputs)
            outputs = outputs.view(outputs.size(0), -1)
            val_features.append(outputs.cpu().numpy())
            val_labels.append(labels.cpu().numpy())
    val_features = np.concatenate(val_features)
    val_labels = np.concatenate(val_labels)

    # Evaluate the classifier on the validation set and extract metrics
    val_predictions = softmax_classifier.predict(val_features)
    accuracy = accuracy_score(val_labels, val_predictions)

    results[fold] = accuracy
    print(f'Fold {fold} Accuracy: {accuracy:.4f}')

# Print the average metrics across all folds
average_accuracy = np.mean(list(results.values()))

print(f'\nK-FOLD CROSS VALIDATION RESULTS FOR {num_folds} FOLDS')
print('--------------------------------')
for fold in results:
    print(f'Fold {fold}: {results[fold]:.4f}')
print(f'Average: {average_accuracy:.4f}')

Fold 1
Fold 1 Accuracy: 0.9246
Fold 2
Fold 2 Accuracy: 0.9287
Fold 3
Fold 3 Accuracy: 0.9242
Fold 4
Fold 4 Accuracy: 0.9158
Fold 5
Fold 5 Accuracy: 0.9313

K-FOLD CROSS VALIDATION RESULTS FOR 2 FOLDS
--------------------------------
Fold 1: 0.9246
Fold 2: 0.9287
Fold 3: 0.9242
Fold 4: 0.9158
Fold 5: 0.9313
Average: 0.9249


### save the model!

In [22]:
joblib.dump(softmax_classifier, 'effnet_softmax.pkl')

['effnet_softmax.pkl']

## EfficientNetB0 + PCA + SVM Classifier Training and Testing
As an extension to our SVM implementation, the paper suggests that applying PCA on the resulting features derives higher accuracy before being loaded into the SVM classifier. We implement this approach below, performing k-fold cross-validation testing on the PCA + SVM classifier, which is trained the on features extracted by our EfficientNetB0 model.

In [14]:
# Store the results of each fold
num_folds = 2
kfold = KFold(n_splits=num_folds, shuffle=True, random_state=231)
results = {}

# Reduce dimensionality to 20 via PCA
n_components = 24

# Create the SVM classifier
pca_classifier = make_pipeline(
    StandardScaler(), 
    PCA(n_components=n_components), 
    SVC(kernel='linear')
)

# K-Fold Cross Validation
for fold, (train_indices, val_indices) in enumerate(kfold.split(train), 1):
    print(f'Fold {fold}')

    # Create data samplers for train and validation sets
    train_sampler = SubsetRandomSampler(train_indices)
    val_sampler = SubsetRandomSampler(val_indices)

    # Create data loaders for train and validation sets
    train_loader = DataLoader(train, batch_size=32, sampler=train_sampler)
    val_loader = DataLoader(train, batch_size=32, sampler=val_sampler)
    
    # Extract features and labels for the training set
    train_features = []
    train_labels = []
    with torch.no_grad():
        for inputs, labels in train_loader:
            outputs = model.forward(inputs)
            outputs = outputs.view(outputs.size(0), -1)
            train_features.append(outputs.cpu().numpy())
            train_labels.append(labels.cpu().numpy())
    train_features = np.concatenate(train_features)
    train_labels = np.concatenate(train_labels)

    # Train the SVM classifier with PCA
    pca_classifier.fit(train_features, train_labels)
    
    # Extract features and labels for the validation set
    val_features = []
    val_labels = []
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model.forward(inputs)
            outputs = outputs.view(outputs.size(0), -1)
            val_features.append(outputs.cpu().numpy())
            val_labels.append(labels.cpu().numpy())
    val_features = np.concatenate(val_features)
    val_labels = np.concatenate(val_labels)

    # Evaluate the classifier on the validation set
    val_predictions = pca_classifier.predict(val_features)
    accuracy = accuracy_score(val_labels, val_predictions)
    results[fold] = accuracy
    print(f'Fold {fold} Accuracy: {accuracy:.4f}')

# Print the average accuracy across all folds
average_accuracy = np.mean(list(results.values()))
print(f'\nK-FOLD CROSS VALIDATION RESULTS FOR {num_folds} FOLDS')
print('--------------------------------')
for fold in results:
    print(f'Fold {fold}: {results[fold]:.4f}')
print(f'Average: {average_accuracy:.4f}')

Fold 1
Fold 1 Accuracy: 0.9033
Fold 2
Fold 2 Accuracy: 0.8985

K-FOLD CROSS VALIDATION RESULTS FOR 2 FOLDS
--------------------------------
Fold 1: 0.9033
Fold 2: 0.8985
Average: 0.9009


In [21]:
joblib.dump(pca_classifier, 'effnet_pca.pkl')

['effnet_pca.pkl']

### Testing and Metrics

Now with our trained models, we will now test with our test set and store metrics for each model. The metrics that we will store are the following:
- Accuracy
- Precision
- Recall
- F1

The metrics are defined in our paper more clearly, but to calculate these we will calculate the the following values:
- True Positive (TP)
- False Positive (FP)
- True Negative (TN)
- False Negative (FN)

We calculate these values below:

In [8]:
# Extract features for training and validation sets
def extract_features(loader, model):
    features_list, labels_list = [], []
    with torch.no_grad():
        for inputs, labels in loader:
            features = model(inputs)
            features = features.view(features.size(0), -1)
            features_list.append(features.cpu().numpy())
            labels_list.append(labels.cpu().numpy())
    return np.concatenate(features_list), np.concatenate(labels_list)

In [13]:
def confusion_matrix_maker(test_labels, test_predictions, classname):
    # Calculate confusion matrix
    cm = confusion_matrix(test_labels, test_predictions, normalize="true")

    # the labels
    label_names = ["ACA", "Healthy", "SCC"]

    # Plot confusion matrix using seaborn
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt="d", xticklabels=label_names, yticklabels=label_names)
    plt.xlabel("Predicted labels")
    plt.ylabel("True labels")
    plt.title("Confusion Matrix: EfficientNetB0 ", classname)
    plt.show()
    plt.savefig(str(classname))

In [ ]:
# Extract Features
test_loader = DataLoader(test, batch_size=32, shuffle=False)
test_features, test_labels = extract_features(test_loader, model)

# List of trained classifiers
classifiers = {
    'SVM': svm_classifier,   # Assume svm_model is already trained
    'Softmax': softmax_classifier,  # Another trained classifier
    'SVM+PCA': pca_classifier   # Another trained classifier
}

# Dictionary to store results
results = {class_name: {} for class_name in classifiers}

# Evaluate each classifier
for class_name, classifier in classifiers.items():
    # Predict using the classifier
    test_predictions = classifier.predict(test_features)
    
    # Calculate metrics
    accuracy = accuracy_score(test_labels, test_predictions)
    precision = precision_score(test_labels, test_predictions, average='weighted')
    recall = recall_score(test_labels, test_predictions, average='weighted')
    f1 = f1_score(test_labels, test_predictions, average='weighted')
    
    # Store the results
    results[class_name]['accuracy'] = accuracy
    results[class_name]['precision'] = precision
    results[class_name]['recall'] = recall
    results[class_name]['f1'] = f1

    # Print the results
    print(f'{class_name} - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}')

    # Make the confusion matrix
    confusion_matrix_maker(test_labels, test_predictions, class_name)

# Print a summary of the results
print('Comparison of Classifiers on Test Set:')
for clf_name in results:
    print(f'{clf_name}: Accuracy={results[clf_name]["accuracy"]:.4f}, Precision={results[clf_name]["precision"]:.4f}, Recall={results[clf_name]["recall"]:.4f}, F1 Score={results[clf_name]["f1"]:.4f}')